In [69]:
from jsonschema import validate, ValidationError
from json import load

In [70]:
def validate_json(json_data, schema_file=schema):
    try:
        validate(instance=json_data, schema=schema)
        return True
    except ValidationError as e:
        return e.message

In [ ]:
from os import path
import json
DIRECTORY = 'batch3'
SCHEMA='schemas/pbcore.schema.json'

with open(SCHEMA, 'r') as schema_file:
    schema = json.load(schema_file)

# with open(path.join(DIRECTORY, 'cpb-aacip_15-29p2p5g8.json'), 'r') as f:
with open('cpb-aacip-15-nc5s756s32.json', 'r') as f:
    json_data = json.load(f)

# json_data

# validate_json(json_data)
validate(json_data, schema)

FileNotFoundError: [Errno 2] No such file or directory: 'batch3/cpb-aacip-15-nc5s756s32.json'

In [72]:
import os
errors = []
for path, subdirs, files in os.walk(DIRECTORY):
    for name in files:
        if name.startswith('cpb-aacip') and name.endswith('.json'):
            with open(os.path.join(path, name), 'r') as f:
                json_data = json.load(f)
                try:
                    validate(json_data, schema)
                    # print(f"{name} is valid.")
                except ValidationError as e:
                    print(f"{name} is invalid: {e.message}")
                    errors.append((name, e.message))


In [73]:
len(errors)

0